In [ ]:
import os
from pathlib import Path

In [ ]:
data_path = Path(os.getcwd()).parent / "data" / "lab_data"
print(data_path)

In [ ]:
import json

sys_prompt = "classify if according to your knowledge observation provided by user is correct or incorrect"

def to_jsonl_line(raw_input: str, correct: bool):
    response = "correct" if correct else "incorrect"
    line = {
        "messages": [
            {"role": "system", "content": "classify if according to your knowledge observation provided by user is correct or incorrect"},
            {"role": "user", "content": f"{raw_input}"},
            {"role": "assistant", "content": response}
        ]
    }
    return json.dumps(line)

In [ ]:
correct = []   
incorrect = []

with open(data_path / "correct.txt", "r") as data:
    for line in data.readlines():
        correct.append(to_jsonl_line(line.strip(), True))

with open(data_path / "incorrect.txt", "r") as data:
    for line in data.readlines():
        correct.append(to_jsonl_line(line.strip(), False))

with open(data_path / "data.jsonl", "w") as jl:
    for line in [*correct, *incorrect]:
        jl.write(f"{line}\n")

In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# assert correct
res = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal:s04e02:AYa1qNCd",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": "94,-14,-46,88"}
    ]
)

msg = res.choices[0].message.content
print(msg)

# assert incorrect
res = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal:s04e02:AYa1qNCd",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": "75,-95,67,46"}
    ]
)

msg = res.choices[0].message.content
print(msg)

In [ ]:
# run on actual data

verify = []
with open(data_path / "verify.txt", "r") as data:
    for line in data.readlines():
        pair = line.strip().split("=")
        verify.append({"key": pair[0], "value": pair[1]})

for i, item in enumerate(verify):
    res = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:personal:s04e02:AYa1qNCd",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": str(item["value"])}
        ]
    )
    outcome = res.choices[0].message.content
    verify[i]["outcome"] = outcome

In [ ]:
answer = [
    item["key"]
    for item in verify
    if item["outcome"] == "correct"
]

In [ ]:
from aidevs3.poligon import send

from dotenv import load_dotenv
load_dotenv()

key = os.environ.get("AG3NTS_API_KEY")
url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/report"

res = send(url, answer=answer, apikey=key, task="research")
print(res)